In [ ]:
import tensorflow as tf
strategy = tf.distribute.MirroredStrategy()
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy.random import randn
from numpy.random import randint
from keras.optimizers import Adam
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Add
from keras.models import load_model
from keras.layers import AveragePooling2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers import Concatenate
from keras.layers import GaussianNoise
from keras.layers import BatchNormalization
from keras.layers import LayerNormalization
from keras.layers import Conv3D
from keras.layers import ConvLSTM3D
from keras.layers import ConvLSTM2D
from keras.layers import TimeDistributed
from keras.initializers import RandomNormal
import keras.backend as K
from sklearn.utils import shuffle

class WGAN(keras.Model):
    def __init__(self, discriminator, generator, Dsteps=5, gp_weight=10.0):
        super(WGAN, self).__init__()

        self.discriminator = discriminator
        self.generator = generator
        self.d_steps = Dsteps
        self.gp_weight = gp_weight

    def compile(self, d_optimizer, g_optimizer, d_loss_fn, g_loss_fn):
        super(WGAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.d_loss_fn = d_loss_fn
        self.g_loss_fn = g_loss_fn
    
    
    def gradient_penalty (self,batch_size, real_images, fake_images):
        alpha = tf.random.uniform([batch_size, 1, 1, 1], minval=0.,maxval=1.)
        diff = fake_images - real_images
        interpolated = real_images + alpha * diff

        with tf.GradientTape() as gp_tape:
            gp_tape.watch(interpolated)
            pred = self.discriminator(interpolated, training=True)

        grads = gp_tape.gradient(pred, [interpolated])[0]
        norm = tf.sqrt(tf.reduce_sum(tf.square(grads), axis=[1, 2, 3]))
        gp = tf.reduce_mean((norm - 1.0) ** 2)
        return gp                             

    def train_step(self, data):
        #if isinstance(data, list):
        lowres_images = data[0]
        real_images=data[1]
        batch_size = tf.shape(real_images)[0]

        for i in range(self.d_steps):
            with tf.GradientTape() as tape:
                # Generate fake images from the latent vector
                fake_images = self.generator(lowres_images, training=True)
                # Get the logits for the fake images
                fake_logits = self.discriminator(fake_images, training=True)
                # Get the logits for the real images
                real_logits = self.discriminator(real_images, training=True)
                # Calculate the discriminator loss using the fake and real image logits
                d_cost = self.d_loss_fn(real_img=real_logits, fake_img=fake_logits)
                # Calculate the gradient penalty
                gp = self.gradient_penalty(batch_size, real_images, fake_images)
                # Add the gradient penalty to the original discriminator loss
                d_loss = d_cost + gp * self.gp_weight
                # Get the gradients w.r.t the discriminator loss
                d_gradient = tape.gradient(d_loss, self.discriminator.trainable_variables)
                # Update the weights of the discriminator using the discriminator optimizer
                self.d_optimizer.apply_gradients(zip(d_gradient, self.discriminator.trainable_variables))
        # Train the generator
        #lowres_images= self.generate_low_res_samples(real_images)
        with tf.GradientTape() as tape:
            # Generate fake images using the generator
            generated_images = self.generator(lowres_images, training=True)
            # Get the discriminator logits for fake images
            gen_img_logits = self.discriminator(generated_images, training=True)
            # Calculate the generator loss
            g_loss = self.g_loss_fn(gen_img_logits)
        # Get the gradients w.r.t the generator loss
        gen_gradient = tape.gradient(g_loss, self.generator.trainable_variables)
        # Update the weights of the generator using the generator optimizer
        self.g_optimizer.apply_gradients(zip(gen_gradient, self.generator.trainable_variables))
        return {"d_loss": d_loss, "g_loss": g_loss, 'batch_size':batch_size}

# define a callback to save keras model after every epoch      
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, gen_loss, critic_loss):
        self.gen_loss=gen_loss
        self.critic_loss=critic_loss
    
    def on_epoch_end(self, epoch, logs={}):
        g_model.save('TempSRgencomb%.1f.keras'%epoch)
        d_model.save('TempSRdisc%.1f.keras'%epoch)
        #pretraining.save('TempSRpretrain%.1f.keras'%epoch)
       # model.save('./MyModel_tf',save_format='tf')
        self.gen_loss.append(logs.get('g_loss'))
        self.critic_loss.append(logs.get('d_loss'))

# list to track the losses for training the WGAN-GP   
gen_loss=[]
critic_loss=[]

# tensorflow method to call all available gpu's for training.
with strategy.scope():
    #define the optimizer for generator and discriminator
    #pretraining_optimizer= keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.2, beta_2=0.9)
    generator_optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.2, beta_2=0.9)
    discriminator_optimizer = keras.optimizers.Adam(learning_rate=0.0001, beta_1=0.2, beta_2=0.9)
    
    def MSE_weighted(y_true,y_pred):
        return K.mean(tf.multiply(tf.square(y_true),tf.square(tf.subtract(y_pred, y_true))))

    #define the pretraining loss function
    def corrector_loss(y_true, y_pred):
        #calculating the soft discritization FSS score with coutoff 0.5 on images scaled [0,1]
        gamma=0.1
        c=10
        cutoff=0.5
        eps = K.epsilon()
        y_true_bi = tf.math.sigmoid( c * ( y_true - cutoff ))
        y_pred_bi = tf.math.sigmoid( c * ( y_pred - cutoff ))
        MSE_n = MSE_weighted(y_true_bi,y_pred_bi) 
        #tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)(y_true_bi, y_pred_bi)
        O_sqimg = tf.keras.layers.Multiply()([y_true_bi, y_true_bi])   
        O_sqvec = tf.keras.layers.Flatten()(O_sqimg)
        M_sqimg = tf.keras.layers.Multiply()([y_pred_bi, y_pred_bi])
        M_sqvec = tf.keras.layers.Flatten()(M_sqimg)
        MSE_ref = tf.math.reduce_mean(O_sqvec + M_sqvec)
        return (tf.math.reduce_mean(tf.keras.losses.huber(y_true, y_pred, delta=0.1)+ gamma*(float(MSE_n) / float(MSE_ref+eps))))
    
                
    #critic loss without the gradient penalty ter
    def discriminator_loss(real_img, fake_img):
        real_loss = tf.reduce_mean(real_img)
        fake_loss = tf.reduce_mean(fake_img)
        return fake_loss - real_loss
    
    #generator loss 
    def generator_loss(fake_img):
        fake_loss= -tf.reduce_mean(fake_img)
        return fake_loss
    
    #loading the model
    d_model = load_model("/kaggle/input/models12epoch/TempSRdisc11.0.keras")
    g_model= load_model("/kaggle/input/models12epoch/TempSRgencomb11.0.keras")
    #pretraining= load_model("/kaggle/input/models12epoch/TempSRpretrain11.0.keras", compile=False)
    cbk = GANMonitor(gen_loss, critic_loss)
    
    #compile the pretraining model using low_res_gen
    #pretraining.compile(optimizer=pretraining_optimizer, loss= corrector_loss)        
    #define the model with generator and critic
    wgan = WGAN(discriminator=d_model, generator=g_model, Dsteps=5)
    # Compile the WGAN model.
    wgan.compile(d_optimizer=discriminator_optimizer, g_optimizer=generator_optimizer, g_loss_fn=generator_loss, d_loss_fn=discriminator_loss,)       
    
#create a low resolution of the observational data to feed the pre-training   
def Pooling(High_Res_Data):
    Avgpool= MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same')
    low1=Avgpool(High_Res_Data)
    low2=Avgpool(low1)
    return low2

#load the data
Input_data=np.load("/kaggle/input/retrainingimdera/Inputfiletemp.npy")
Highres_data=np.load("/kaggle/input/retrainingimdera/Highresfiletemp.npy")
Low_Res_Data, High_Res_Data= shuffle(Input_data, Highres_data)
print(Low_Res_Data.shape,High_Res_Data.shape)

#define the batch size and epochs
pre_epoch=10
pre_batch_size=128
training_epoch =20
training_batch_size=128
total_samples=High_Res_Data.shape[0]

#maintain consistent number of samples per epoch
stpe_wgan=total_samples//training_batch_size
stpe_pretrain=total_samples//pre_batch_size
#create a low resolution of the observational data set
Maxpooled_data=Pooling(High_Res_Data)

#Train the model.
#pretraining.fit(Low_Res_Data, Maxpooled_data,batch_size=pre_batch_size,epochs=pre_epoch,steps_per_epoch=stpe_pretrain, verbose=2)
wgan.fit(Low_Res_Data, High_Res_Data, batch_size=training_batch_size, epochs=training_epoch,callbacks=[cbk],steps_per_epoch=stpe_wgan, verbose=2)



In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from numpy import expand_dims
import numpy.ma as ma
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.cm as mtpltcm
from mpl_toolkits.mplot3d import Axes3D
import random
from netCDF4 import Dataset
import cartopy.crs as ccrs
from tensorflow import keras
from keras.layers import AveragePooling2D
from keras.layers import MaxPooling2D
from keras.layers import Add
from keras.models import load_model
import numpy.ma as ma
import os
import glob
np.set_printoptions(threshold=np.inf)

INPUT=np.load("/kaggle/input/training-error/Inputfiletemp2017-2019.npy")
OUTPUT=np.load("/kaggle/input/training-error/Highresfiletemp2017-2019.npy")
files=glob.glob(os.path.join("/kaggle/input/gen-epochs", "*.keras"))
models = [load_model(file) for file in files]
RMSE=tf.keras.metrics.RootMeanSquaredError()
print(INPUT.shape, OUTPUT.shape) 
RMSEMean=[]
RMSESD=[]
for j in range(len(models)):
    print("model",j,"starting")
    Error=[]
    for i in range(INPUT.shape[0]):
        gen=INPUT[i]
        era=OUTPUT[i]
        era=era*51
        era=np.reshape(era, (48,48))
        gen=np.reshape(gen, (1, 12, 12, 1))
        OUT = models[j].predict(gen, verbose=0)
        OUT=np.reshape(OUT,(48,48))
        OUT=OUT*51
        Error.append(RMSE(OUT,era))
    RMSEMean.append(np.mean(np.array(Error)))
    RMSESD.append(np.std(np.array(Error)))

np.save("Trainerrormean", np.array(RMSEMean))
np.save("Trainerrorsd", np.array(RMSESD))